In [1]:
# This script creates a ton of features, then trains an ensemble of XGBoost 
# regressor, Lasso regressor, ElasticNet and KernelRidge.
#
# It borrows ideas from lots of other people's scripts, including:
# https://www.kaggle.com/humananalog/house-prices-advanced-regression-techniques/xgboost-lasso
# https://www.kaggle.com/klyusba/house-prices-advanced-regression-techniques/lasso-model-for-regression-problem/notebook
# https://www.kaggle.com/juliencs/house-prices-advanced-regression-techniques/a-study-on-regression-applied-to-the-ames-dataset/
# https://www.kaggle.com/apapiu/house-prices-advanced-regression-techniques/regularized-linear-models

import datetime
import numpy as np
import pandas as pd

# The error metric: RMSE on the log of the sale prices.
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import xgboost as xgb
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.kernel_ridge import KernelRidge
from sklearn.base import BaseEstimator, RegressorMixin

from sklearn.preprocessing import LabelEncoder
from scipy.stats import skew

In [2]:
# Load the data.
base_path='D:/kaggle/regression/'
train_df = pd.read_csv(base_path+'train.csv',index_col='Id')
train_target=train_df['SalePrice']
train_df=train_df.drop(['SalePrice'],axis=1)
test_df = pd.read_csv(base_path+'test.csv',index_col='Id')

In [3]:
print train_df.index
print test_df.index

print train_df.shape
print test_df.shape

print len(train_target)

Int64Index([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
            ...
            1451, 1452, 1453, 1454, 1455, 1456, 1457, 1458, 1459, 1460],
           dtype='int64', name=u'Id', length=1460)
Int64Index([1461, 1462, 1463, 1464, 1465, 1466, 1467, 1468, 1469, 1470,
            ...
            2910, 2911, 2912, 2913, 2914, 2915, 2916, 2917, 2918, 2919],
           dtype='int64', name=u'Id', length=1459)
(1460, 79)
(1459, 79)
1460


In [4]:
# Fill the NA in the data
def fill_na_by_cat(df):
    alldata=df.copy(deep=True)
    
    # Fill some columns first
    alldata['MSSubClass'].fillna(np.nanmedian(df['MSSubClass']), inplace=True)
    alldata['MSZoning'].fillna('RL', inplace=True)
    alldata['Exterior1st'].fillna('Other', inplace=True)
    alldata['Exterior2nd'].fillna('Other', inplace=True)
    alldata['Exterior1st'].fillna('Other', inplace=True)
    alldata['MasVnrType'].fillna('None', inplace=True)
    alldata['MasVnrType'].fillna('None', inplace=True)
    alldata['SaleType'].fillna('Oth', inplace=True)
    
    # The years are really tricky
    alldata['GarageYrBlt'].fillna(1899, inplace=True)
    alldata['YearBuilt'].fillna(1899, inplace=True)
    alldata['YearRemodAdd'].fillna(2017, inplace=True)
    alldata['YrSold'].fillna(2017, inplace=True)
    
    # Locate the NA columns
    nacount_category=np.array(alldata.columns[((alldata.dtypes=='int64')|(alldata.dtypes=='float32')|(alldata.dtypes=='float64')| (alldata.dtypes=='object')) & (pd.isnull(alldata).sum()>0)])
    print 'Columns that contain NA:',
    print nacount_category
    
    # Define a few categories
    category=np.array(alldata.columns[((alldata.dtypes=='int64') | (alldata.dtypes=='object'))])
    Bsmtset=set(['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2'])
    MasVnrset=set(['MasVnrType'])
    Garageset=set(['GarageType','GarageYrBlt','GarageFinish','GarageQual','GarageCond'])
    Fireplaceset=set(['FireplaceQu'])
    Poolset=set(['PoolQC'])
    Emptyset=set(['Fence','MiscFeature','Alley'])
    NAset=set(['MSZoning','Street','LotShape','LandContour','Utilities','LotConfig','LandSlope',
               'Neighborhood','Condition1','Condition2','BldgType','HouseStyle','RoofStyle',
               'RoofMatl','Exterior1st','Exterior2nd','ExterQual','ExterCond','Foundation','Heating','HeatingQC',
               'CentralAir','Electrical','KitchenQual','Functional','PavedDrive','SaleType','SaleCondition'])
    Zeroset=['LotArea','LotFrontage','MasVnrArea','BsmtFinSF1','BsmtFinSF2',
            'BsmtUnfSF','TotalBsmtSF','BsmtFullBath','BsmtHalfBath','GarageCars','GarageArea',
            'KitchenQual','Functional','LotArea','BsmtFullBath','BsmtHalfBath','PoolArea',
            'Fireplaces','WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch',
             'MiscVal'
            ] 
    Medianset=['MSSubClass','1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea','BedroomAbvGr',
              'KitchenAbvGr','TotRmsAbvGrd','FullBath','HalfBath']
    Averageset=['OverallQual','OverallCond']
    
    # Fill based on category
    for i in nacount_category:
        if i in Bsmtset:
            alldata.ix[pd.isnull(alldata.ix[:,i]) & (alldata['TotalBsmtSF']==0),i]='Empty'
            alldata.ix[pd.isnull(alldata.ix[:,i]),i]=alldata.ix[:,i].value_counts().index[0]
        elif i in MasVnrset:
            alldata.ix[pd.isnull(alldata.ix[:,i]) & (alldata['MasVnrArea']==0),i]='Empty'
            alldata.ix[pd.isnull(alldata.ix[:,i]),i]=alldata.ix[:,i].value_counts().index[0]
        elif i in Garageset:
            alldata.ix[pd.isnull(alldata.ix[:,i]) & (alldata['GarageArea']==0),i]='Empty'
            alldata.ix[pd.isnull(alldata.ix[:,i]),i]=alldata.ix[:,i].value_counts().index[0]
        elif i in Fireplaceset:
            alldata.ix[pd.isnull(alldata.ix[:,i]) & (alldata['Fireplaces']==0),i]='Empty'
            alldata.ix[pd.isnull(alldata.ix[:,i]),i]=alldata.ix[:,i].value_counts().index[0]
        elif i in Poolset:
            alldata.ix[pd.isnull(alldata.ix[:,i]) & (alldata['PoolArea']==0),i]='Empty'
            alldata.ix[pd.isnull(alldata.ix[:,i]),i]=alldata.ix[:,i].value_counts().index[0]
        elif i in Emptyset:
            alldata.ix[pd.isnull(alldata.ix[:,i]),i]='Empty'
        elif i in NAset:
            alldata.ix[pd.isnull(alldata.ix[:,i]),i]='NA'
        elif i in Zeroset:
            alldata.ix[pd.isnull(alldata.ix[:,i]),i]=0
        elif i in Averageset:
            alldata.ix[pd.isnull(alldata.ix[:,i]),i]=5
        elif i in Medianset:
#             print 'Filling column ',i,' with median'
            alldata.ix[pd.isnull(alldata.ix[:,i]),i]=np.nanmedian(alldata[i])
        else:
            if alldata[i].dtype == 'object':
                print i
#                 print 'Column ',i,' is of type object. Leave it there.'
            else:
                print 'Column ',i,' still needs handling'
#             alldata.ix[pd.isnull(alldata.ix[:,i]),i]=alldata.ix[:,i].value_counts().index[0] # This fills NA with the 1st value, not the best way
            
    # Valudate before returning
    nacount_category=np.array(alldata.columns[((alldata.dtypes=='int64')|(alldata.dtypes=='float32')|(alldata.dtypes=='float64')| (alldata.dtypes=='object')) & (pd.isnull(alldata).sum()>0)])
    print 'Columns that still contain NA:',
    print nacount_category
    
    return alldata

In [5]:
train_df['Electrical'].dtype == 'object'

True

In [36]:
# Cbeck the type of column
targets=['Electrical','3SsnPorch', 'BsmtFullBath', 'BsmtUnfSF', 'OpenPorchSF', '2ndFlrSF', 'HalfBath', 'OverallCond', 'LotArea', 'GrLivArea', 'YearRemodAdd', 'MasVnrArea', '1stFlrSF', 'Fireplaces', 'LotFrontage', 'MiscVal', 'OverallQual', 'KitchenAbvGr', 'WoodDeckSF', 'BsmtHalfBath', 'TotalBsmtSF', 'TotRmsAbvGrd', 'EnclosedPorch', 'FullBath', 'YrSold', 'MSSubClass', 'YearBuilt', 'BsmtFinSF1', 'BsmtFinSF2', 'PoolArea', 'ScreenPorch', 'BedroomAbvGr', 'LowQualFinSF', 'GarageArea', 'GarageCars']
for t in targets:
    print t,' type: ',train_df[t].dtype

Electrical  type:  object
3SsnPorch  type:  int64
BsmtFullBath  type:  int64
BsmtUnfSF  type:  int64
OpenPorchSF  type:  int64
2ndFlrSF  type:  int64
HalfBath  type:  int64
OverallCond  type:  int64
LotArea  type:  int64
GrLivArea  type:  int64
YearRemodAdd  type:  int64
MasVnrArea  type:  float64
1stFlrSF  type:  int64
Fireplaces  type:  int64
LotFrontage  type:  float64
MiscVal  type:  int64
OverallQual  type:  int64
KitchenAbvGr  type:  int64
WoodDeckSF  type:  int64
BsmtHalfBath  type:  int64
TotalBsmtSF  type:  int64
TotRmsAbvGrd  type:  int64
EnclosedPorch  type:  int64
FullBath  type:  int64
YrSold  type:  int64
MSSubClass  type:  int64
YearBuilt  type:  int64
BsmtFinSF1  type:  int64
BsmtFinSF2  type:  int64
PoolArea  type:  int64
ScreenPorch  type:  int64
BedroomAbvGr  type:  int64
LowQualFinSF  type:  int64
GarageArea  type:  int64
GarageCars  type:  int64


In [6]:
# Fill NA columns
train_filled=fill_na_by_cat(train_df)
train_filled.index=train_df.index
test_filled=fill_na_by_cat(test_df)
test_filled.index=test_df.index

Columns that contain NA: ['LotFrontage' 'Alley' 'MasVnrArea' 'BsmtQual' 'BsmtCond' 'BsmtExposure'
 'BsmtFinType1' 'BsmtFinType2' 'Electrical' 'FireplaceQu' 'GarageType'
 'GarageFinish' 'GarageQual' 'GarageCond' 'PoolQC' 'Fence' 'MiscFeature']
Columns that still contain NA: []
Columns that contain NA: ['LotFrontage' 'Alley' 'Utilities' 'MasVnrArea' 'BsmtQual' 'BsmtCond'
 'BsmtExposure' 'BsmtFinType1' 'BsmtFinSF1' 'BsmtFinType2' 'BsmtFinSF2'
 'BsmtUnfSF' 'TotalBsmtSF' 'BsmtFullBath' 'BsmtHalfBath' 'KitchenQual'
 'Functional' 'FireplaceQu' 'GarageType' 'GarageFinish' 'GarageCars'
 'GarageArea' 'GarageQual' 'GarageCond' 'PoolQC' 'Fence' 'MiscFeature']
Columns that still contain NA: []


In [7]:
# Check for nan in numeric features
def check_nan(df,features):
    empty_list=[]
    for f in features:
        empty_index=df[df[f].isnull()].index
        if len(empty_index) > 0:
            print 'Feature ',f,'of type ',df[f].dtype,' contains NAN'
            empty_list.append(f)
            print empty_index
    return empty_list

In [8]:
# Check if there are any NA
print 'Checking NA before anything starts'
numeric_features = test_filled.dtypes[test_filled.dtypes != 'object'].index
print 'Numeric features: ',
print numeric_features

print 'Checking training set'
empty_columns_train=check_nan(train_filled,numeric_features)
print 'Checking test set'
empty_columns_test=check_nan(test_filled,numeric_features)

# Output the set that still needs handling
still_na=list(set(empty_columns_train) | set(empty_columns_test))
print 'Columns that still needs NA handling'
print still_na

Checking NA before anything starts
Numeric features:  Index([u'MSSubClass', u'LotFrontage', u'LotArea', u'OverallQual',
       u'OverallCond', u'YearBuilt', u'YearRemodAdd', u'MasVnrArea',
       u'BsmtFinSF1', u'BsmtFinSF2', u'BsmtUnfSF', u'TotalBsmtSF', u'1stFlrSF',
       u'2ndFlrSF', u'LowQualFinSF', u'GrLivArea', u'BsmtFullBath',
       u'BsmtHalfBath', u'FullBath', u'HalfBath', u'BedroomAbvGr',
       u'KitchenAbvGr', u'TotRmsAbvGrd', u'Fireplaces', u'GarageYrBlt',
       u'GarageCars', u'GarageArea', u'WoodDeckSF', u'OpenPorchSF',
       u'EnclosedPorch', u'3SsnPorch', u'ScreenPorch', u'PoolArea', u'MiscVal',
       u'MoSold', u'YrSold'],
      dtype='object')
Checking training set
Checking test set
Columns that still needs NA handling
[]


In [1]:
# Convert catagorical to numerical
def factorize(df, factor_df, column):
    factor_df[column] = df[column]
    
    # Check if there are still NA
    print 'Checking NA'
    print factor_df.loc[factor_df[column].isnull()]
    
    le.fit(factor_df[column].unique())
    factor_df[column] = le.transform(factor_df[column])
    return factor_df

In [10]:
# Print unique values in the column
def check_unique_values(column):
    print column.unique

In [11]:
# Combine all the (numerical) features into one big DataFrame. We don't add 
# the one-hot encoded variables here yet, that happens later on.
def munge(df):
    all_df = pd.DataFrame(index = df.index)
    print 'Target DF has index column ',all_df.index
   
    all_df['LotFrontage'] = df['LotFrontage']   
#     for key, group in lot_frontage_by_neighborhood:
#         idx = (df['Neighborhood'] == key) & (df['LotFrontage'].isnull())
#         all_df.loc[idx, 'LotFrontage'] = group.median()    

    all_df['LotArea'] = df['LotArea']

    all_df['MasVnrArea'] = df['MasVnrArea']
#     all_df['MasVnrArea'].fillna(0, inplace=True)
   
    all_df['BsmtFinSF1'] = df['BsmtFinSF1']
#     all_df['BsmtFinSF1'].fillna(0, inplace=True)

    all_df['BsmtFinSF2'] = df['BsmtFinSF2']
#     all_df['BsmtFinSF2'].fillna(0, inplace=True)

    all_df['BsmtUnfSF'] = df['BsmtUnfSF']
#     all_df['BsmtUnfSF'].fillna(0, inplace=True)

    all_df['TotalBsmtSF'] = df['TotalBsmtSF']
#     all_df['TotalBsmtSF'].fillna(0, inplace=True)

    all_df['1stFlrSF'] = df['1stFlrSF']
    all_df['2ndFlrSF'] = df['2ndFlrSF']
    all_df['GrLivArea'] = df['GrLivArea']
    
    all_df['GarageArea'] = df['GarageArea']
#     all_df['GarageArea'].fillna(0, inplace=True)

    all_df['WoodDeckSF'] = df['WoodDeckSF']
    all_df['OpenPorchSF'] = df['OpenPorchSF']
    all_df['EnclosedPorch'] = df['EnclosedPorch']
    all_df['3SsnPorch'] = df['3SsnPorch']
    all_df['ScreenPorch'] = df['ScreenPorch']
    
    all_df['BsmtFullBath'] = df['BsmtFullBath']
#     all_df['BsmtFullBath'].fillna(0, inplace=True)

    all_df['BsmtHalfBath'] = df['BsmtHalfBath']
#     all_df['BsmtHalfBath'].fillna(0, inplace=True)

    all_df['FullBath'] = df['FullBath'] 
    all_df['HalfBath'] = df['HalfBath'] 
    all_df['BedroomAbvGr'] = df['BedroomAbvGr'] 
    all_df['KitchenAbvGr'] = df['KitchenAbvGr'] 
    all_df['TotRmsAbvGrd'] = df['TotRmsAbvGrd'] 
    all_df['Fireplaces'] = df['Fireplaces'] 

    all_df['GarageCars'] = df['GarageCars']
#     all_df['GarageCars'].fillna(0, inplace=True)

    all_df['CentralAir'] = (df['CentralAir'] == 'Y') * 1.0
   
    all_df['OverallQual'] = df['OverallQual']
    all_df['OverallCond'] = df['OverallCond']

    # Quality measurements are stored as text but we can convert them to 
    # numbers where a higher number means higher quality.

    qual_dict = {None: 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5,'Empty':0,'NA':0}
    all_df['ExterQual'] = df['ExterQual'].map(qual_dict).astype(int)
    all_df['ExterCond'] = df['ExterCond'].map(qual_dict).astype(int)
    
    all_df['BsmtQual'] = df['BsmtQual'].map(qual_dict).astype(int)
    all_df['BsmtCond'] = df['BsmtCond'].map(qual_dict).astype(int)
    all_df['HeatingQC'] = df['HeatingQC'].map(qual_dict).astype(int)
    all_df['KitchenQual'] = df['KitchenQual'].map(qual_dict).astype(int)
    all_df['FireplaceQu'] = df['FireplaceQu'].map(qual_dict).astype(int)
    all_df['GarageQual'] = df['GarageQual'].map(qual_dict).astype(int)
    all_df['GarageCond'] = df['GarageCond'].map(qual_dict).astype(int)

    all_df['BsmtExposure'] = df['BsmtExposure'].map(
        {None: 0, 'No': 1, 'Mn': 2, 'Av': 3, 'Gd': 4,'Empty':0}).astype(int)

    bsmt_fin_dict = {None: 0, 'Unf': 1, 'LwQ': 2, 'Rec': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6,'Empty':0}
    all_df['BsmtFinType1'] = df['BsmtFinType1'].map(bsmt_fin_dict).astype(int)
    all_df['BsmtFinType2'] = df['BsmtFinType2'].map(bsmt_fin_dict).astype(int)

    all_df['Functional'] = df['Functional'].map(
        {None: 0, 'Sal': 1, 'Sev': 2, 'Maj2': 3, 'Maj1': 4, 
         'Mod': 5, 'Min2': 6, 'Min1': 7, 'Typ': 8,'Empty':0,'NA':0}).astype(int)

    all_df['GarageFinish'] = df['GarageFinish'].map(
        {None: 0, 'Unf': 1, 'RFn': 2, 'Fin': 3,'Empty':0,'NA':0}).astype(int)

    all_df['Fence'] = df['Fence'].map(
        {None: 0, 'MnWw': 1, 'GdWo': 2, 'MnPrv': 3, 'GdPrv': 4,'Empty':0,'NA':0}).astype(int)

    all_df['YearBuilt'] = df['YearBuilt']
    all_df['YearRemodAdd'] = df['YearRemodAdd']

    all_df['GarageYrBlt'] = df['GarageYrBlt']
#     all_df['GarageYrBlt'].fillna(0.0, inplace=True)

    all_df['MoSold'] = df['MoSold']
    all_df['YrSold'] = df['YrSold']
    
    all_df['LowQualFinSF'] = df['LowQualFinSF']
    all_df['MiscVal'] = df['MiscVal']

    all_df['PoolQC'] = df['PoolQC'].map(qual_dict).astype(int)

    all_df['PoolArea'] = df['PoolArea']
#     all_df['PoolArea'].fillna(0, inplace=True)
    
    # Add categorical features as numbers too. It seems to help a bit.
    all_df = factorize(df, all_df, 'MSSubClass')
    all_df = factorize(df, all_df, 'MSZoning')
    all_df = factorize(df, all_df, 'LotConfig')
    all_df = factorize(df, all_df, 'Neighborhood')
    all_df = factorize(df, all_df, 'Condition1')
    all_df = factorize(df, all_df, 'BldgType')
    all_df = factorize(df, all_df, 'HouseStyle')
    all_df = factorize(df, all_df, 'RoofStyle')
    all_df = factorize(df, all_df, 'Exterior1st')
    all_df = factorize(df, all_df, 'Exterior2nd')
    all_df = factorize(df, all_df, 'MasVnrType')
    all_df = factorize(df, all_df, 'Foundation')
    all_df = factorize(df, all_df, 'SaleType')
    all_df = factorize(df, all_df, 'SaleCondition')

    # IR2 and IR3 don't appear that often, so just make a distinction
    # between regular and irregular.
    all_df['IsRegularLotShape'] = (df['LotShape'] == 'Reg') * 1

    # Most properties are level; bin the other possibilities together
    # as 'not level'.
    all_df['IsLandLevel'] = (df['LandContour'] == 'Lvl') * 1

    # Most land slopes are gentle; treat the others as 'not gentle'.
    all_df['IsLandSlopeGentle'] = (df['LandSlope'] == 'Gtl') * 1

    # Most properties use standard circuit breakers.
    all_df['IsElectricalSBrkr'] = (df['Electrical'] == 'SBrkr') * 1

    # About 2/3rd have an attached garage.
    all_df['IsGarageDetached'] = (df['GarageType'] == 'Detchd') * 1

    # Most have a paved drive. Treat dirt/gravel and partial pavement
    # as 'not paved'.
    all_df['IsPavedDrive'] = (df['PavedDrive'] == 'Y') * 1

    # The only interesting 'misc. feature' is the presence of a shed.
    all_df['HasShed'] = (df['MiscFeature'] == 'Shed') * 1.  

    # If YearRemodAdd != YearBuilt, then a remodeling took place at some point.
    all_df['Remodeled'] = (all_df['YearRemodAdd'] != all_df['YearBuilt']) * 1
    
    # Did a remodeling happen in the year the house was sold?
    all_df['RecentRemodel'] = (all_df['YearRemodAdd'] == all_df['YrSold']) * 1
    
    # Was this house sold in the year it was built?
    all_df['VeryNewHouse'] = (all_df['YearBuilt'] == all_df['YrSold']) * 1

    all_df['Has2ndFloor'] = (all_df['2ndFlrSF'] == 0) * 1
    all_df['HasMasVnr'] = (all_df['MasVnrArea'] == 0) * 1
    all_df['HasWoodDeck'] = (all_df['WoodDeckSF'] == 0) * 1
    all_df['HasOpenPorch'] = (all_df['OpenPorchSF'] == 0) * 1
    all_df['HasEnclosedPorch'] = (all_df['EnclosedPorch'] == 0) * 1
    all_df['Has3SsnPorch'] = (all_df['3SsnPorch'] == 0) * 1
    all_df['HasScreenPorch'] = (all_df['ScreenPorch'] == 0) * 1

    # These features actually lower the score a little.
    # all_df['HasBasement'] = df['BsmtQual'].isnull() * 1
    # all_df['HasGarage'] = df['GarageQual'].isnull() * 1
    # all_df['HasFireplace'] = df['FireplaceQu'].isnull() * 1
    # all_df['HasFence'] = df['Fence'].isnull() * 1

    # Months with the largest number of deals may be significant.
    all_df['HighSeason'] = df['MoSold'].replace( 
        {1: 0, 2: 0, 3: 0, 4: 1, 5: 1, 6: 1, 7: 1, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0})

    all_df['NewerDwelling'] = df['MSSubClass'].replace(
        {20: 1, 30: 0, 40: 0, 45: 0,50: 0, 60: 1, 70: 0, 75: 0, 80: 0, 85: 0,
         90: 0, 120: 1, 150: 0, 160: 0, 180: 0, 190: 0})   
    
    all_df.loc[df.Neighborhood == 'NridgHt', 'Neighborhood_Good'] = 1
    all_df.loc[df.Neighborhood == 'Crawfor', 'Neighborhood_Good'] = 1
    all_df.loc[df.Neighborhood == 'StoneBr', 'Neighborhood_Good'] = 1
    all_df.loc[df.Neighborhood == 'Somerst', 'Neighborhood_Good'] = 1
    all_df.loc[df.Neighborhood == 'NoRidge', 'Neighborhood_Good'] = 1
    all_df['Neighborhood_Good'].fillna(0, inplace=True)

    all_df['SaleCondition_PriceDown'] = df.SaleCondition.replace(
        # {'Abnorml': 1, 'Alloca': 1, 'AdjLand': 1, 'Family': 1, 'Normal': 0, 'Partial': 0})
        {'Abnorml': 1, 'Alloca': 2, 'AdjLand': 3, 'Family': 4, 'Normal': 5, 'Partial': 0})

    # House completed before sale or not
    all_df['BoughtOffPlan'] = df.SaleCondition.replace(
        {'Abnorml' : 0, 'Alloca' : 0, 'AdjLand' : 0, 'Family' : 0, 'Normal' : 0, 'Partial' : 1})
    
    # all_df['BadHeating'] = df.HeatingQC.replace(
    #     {'Ex': 0, 'Gd': 0, 'TA': 0, 'Fa': 1, 'Po': 1})
    all_df['BadHeating'] = df.HeatingQC.replace(
        {'Ex': 0, 'Gd': 1, 'TA': 2, 'Fa': 3, 'Po': 4})

    area_cols = ['LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
                 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'GarageArea', 'WoodDeckSF', 
                 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'LowQualFinSF', 'PoolArea' ]
    all_df['TotalArea'] = all_df[area_cols].sum(axis=1)

    all_df['TotalArea1st2nd'] = all_df['1stFlrSF'] + all_df['2ndFlrSF']

    all_df['Age'] = 2010 - all_df['YearBuilt']
    all_df['TimeSinceSold'] = 2010 - all_df['YrSold']

    # If commented - a little bit worse on LB but better in CV
#     all_df['SeasonSold'] = all_df['MoSold'].map({12:0, 1:0, 2:0, 3:1, 4:1, 5:1, 
#                                                   6:2, 7:2, 8:2, 9:3, 10:3, 11:3,}).astype(int)
    
    all_df['YearsSinceRemodel'] = all_df['YrSold'] - all_df['YearRemodAdd']
    all_df['SimplOverallQual'] = all_df.OverallQual
    all_df['SimplOverallCond'] = all_df.OverallCond
    all_df['SimplPoolQC'] = all_df.PoolQC
    all_df['SimplGarageCond'] = all_df.GarageCond
    all_df['SimplGarageQual'] = all_df.GarageQual
    all_df['SimplFireplaceQu'] = all_df.FireplaceQu
    all_df['SimplFireplaceQu'] = all_df.FireplaceQu
    all_df['SimplFunctional'] = all_df.Functional
    all_df['SimplKitchenQual'] = all_df.KitchenQual
    all_df['SimplHeatingQC'] = all_df.HeatingQC
    all_df['SimplBsmtFinType1'] = all_df.BsmtFinType1
    all_df['SimplBsmtFinType2'] = all_df.BsmtFinType2
    all_df['SimplBsmtCond'] = all_df.BsmtCond
    all_df['SimplBsmtQual'] = all_df.BsmtQual
    all_df['SimplExterCond'] = all_df.ExterCond
    all_df['SimplExterQual'] = all_df.ExterQual
    
    # Bin by neighborhood (a little arbitrarily). Values were computed by: 
    # train_df['SalePrice'].groupby(train_df['Neighborhood']).median().sort_values()
    neighborhood_map = {
        'MeadowV' : 0,  #  88000
        'IDOTRR' : 1,   # 103000
        'BrDale' : 1,   # 106000
        'OldTown' : 1,  # 119000
        'Edwards' : 1,  # 119500
        'BrkSide' : 1,  # 124300
        'Sawyer' : 1,   # 135000
        'Blueste' : 1,  # 137500
        'SWISU' : 2,    # 139500
        'NAmes' : 2,    # 140000
        'NPkVill' : 2,  # 146000
        'Mitchel' : 2,  # 153500
        'SawyerW' : 2,  # 179900
        'Gilbert' : 2,  # 181000
        'NWAmes' : 2,   # 182900
        'Blmngtn' : 2,  # 191000
        'CollgCr' : 2,  # 197200
        'ClearCr' : 3,  # 200250
        'Crawfor' : 3,  # 200624
        'Veenker' : 3,  # 218000
        'Somerst' : 3,  # 225500
        'Timber' : 3,   # 228475
        'StoneBr' : 4,  # 278000
        'NoRidge' : 4,  # 290000
        'NridgHt' : 4,  # 315000
    }

    all_df['NeighborhoodBin'] = df['Neighborhood'].map(neighborhood_map)
    return all_df

In [71]:
# Find if there are any value that are not covered by the conversion
qual_dict = {None: 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5,'Empty':0}
for i in test_filled['KitchenQual']:
    if i not in qual_dict:
        print i

NA


In [72]:
d={None: 0, 'Sal': 1, 'Sev': 2, 'Maj2': 3, 'Maj1': 4, 
         'Mod': 5, 'Min2': 6, 'Min1': 7, 'Typ': 8,'Empty':0}
for i in test_filled['Functional']:
    if i not in d:
        print i

NA
NA


In [12]:
# For imputing missing values: fill in missing LotFrontage values by the median
# LotFrontage of the neighborhood.
lot_frontage_by_neighborhood = train_df['LotFrontage'].groupby(train_df['Neighborhood'])

In [13]:
# Check NAN -- example code
train_filled.loc[train_filled.BsmtQual.isnull()]
test_filled.loc[test_filled.BsmtQual.isnull()]

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,


In [14]:
# Print columns that contains NAN
def na_columns(df):
    for k,v in df.isnull().any().iteritems():
        if v:
            print k

In [15]:
na_columns(train_filled)
na_columns(test_filled)

In [16]:
# Used to convert categorical features into ordinal numbers.
# (There's probably an easier way to do this, but it works.)
le = LabelEncoder()

test_df_munged = munge(test_filled)
test_df_munged.index=test_filled.index
train_df_munged = munge(train_filled)
train_df_munged.index=train_filled.index

print('Training set size:', train_df_munged.shape)
print('Training set index:',train_df_munged.index)
print('Test set size:', test_df_munged.shape)
print('Test set index:',test_df_munged.index)


Target DF has index column  Int64Index([1461, 1462, 1463, 1464, 1465, 1466, 1467, 1468, 1469, 1470,
            ...
            2910, 2911, 2912, 2913, 2914, 2915, 2916, 2917, 2918, 2919],
           dtype='int64', name=u'Id', length=1459)
Checking NA
Empty DataFrame
Columns: [LotFrontage, LotArea, MasVnrArea, BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, 1stFlrSF, 2ndFlrSF, GrLivArea, GarageArea, WoodDeckSF, OpenPorchSF, EnclosedPorch, 3SsnPorch, ScreenPorch, BsmtFullBath, BsmtHalfBath, FullBath, HalfBath, BedroomAbvGr, KitchenAbvGr, TotRmsAbvGrd, Fireplaces, GarageCars, CentralAir, OverallQual, OverallCond, ExterQual, ExterCond, BsmtQual, BsmtCond, HeatingQC, KitchenQual, FireplaceQu, GarageQual, GarageCond, BsmtExposure, BsmtFinType1, BsmtFinType2, Functional, GarageFinish, Fence, YearBuilt, YearRemodAdd, GarageYrBlt, MoSold, YrSold, LowQualFinSF, MiscVal, PoolQC, PoolArea, MSSubClass]
Index: []

[0 rows x 53 columns]
Checking NA
Empty DataFrame
Columns: [LotFrontage, LotArea, Mas

In [17]:
# Copy NeighborhoodBin into a temporary DataFrame because we want to use the
# unscaled version later on (to one-hot encode it). 
neighborhood_bin_train = pd.DataFrame(index = train_df.index)
neighborhood_bin_train['NeighborhoodBin'] = train_df_munged['NeighborhoodBin']
neighborhood_bin_test = pd.DataFrame(index = test_df.index)
neighborhood_bin_test['NeighborhoodBin'] = test_df_munged['NeighborhoodBin']

In [18]:
# Before transforming, check if there are any NA
print 'Checking NA before anything starts'
numeric_features = test_df_munged.dtypes[test_df_munged.dtypes != 'object'].index
print 'Numeric features: ',
print numeric_features

print 'Checking training set'
empty_columns_train=check_nan(train_df_munged,numeric_features)
print 'Checking test set'
empty_columns_test=check_nan(test_df_munged,numeric_features)

# Output the set that still needs handling
still_na=list(set(empty_columns_train) | set(empty_columns_test))
print 'Columns that still needs NA handling'
print still_na

Checking NA before anything starts
Numeric features:  Index([u'LotFrontage', u'LotArea', u'MasVnrArea', u'BsmtFinSF1', u'BsmtFinSF2',
       u'BsmtUnfSF', u'TotalBsmtSF', u'1stFlrSF', u'2ndFlrSF', u'GrLivArea',
       ...
       u'SimplFunctional', u'SimplKitchenQual', u'SimplHeatingQC',
       u'SimplBsmtFinType1', u'SimplBsmtFinType2', u'SimplBsmtCond',
       u'SimplBsmtQual', u'SimplExterCond', u'SimplExterQual',
       u'NeighborhoodBin'],
      dtype='object', length=110)
Checking training set
Checking test set
Columns that still needs NA handling
[]


In [19]:
# Transform the skewed numeric features by taking log(feature + 1).
# This will make the features more normal.
skewed = train_df_munged[numeric_features].apply(lambda x: skew(x.dropna().astype(float)))
skewed = skewed[skewed > 0.8]
skewed = skewed.index

train_df_munged[skewed] = np.log1p(train_df_munged[skewed])
test_df_munged[skewed] = np.log1p(test_df_munged[skewed])


In [20]:
# Additional processing: scale the data.   
from sklearn.preprocessing import StandardScaler, RobustScaler
scaler = StandardScaler()
scaler.fit(train_df_munged[numeric_features])

scaled = scaler.transform(train_df_munged[numeric_features])
for i, col in enumerate(numeric_features):
    train_df_munged[col] = scaled[:, i]

scaled = scaler.transform(test_df_munged[numeric_features])
for i, col in enumerate(numeric_features):
    test_df_munged[col] = scaled[:, i]

In [21]:
print train_df_munged.shape
print test_df_munged.shape

(1460, 110)
(1459, 110)


In [22]:
train_transformed=train_df_munged.copy(deep=True)
test_transformed=test_df_munged.copy(deep=True)

In [23]:
# Convert categorical features using one-hot encoding.
def onehot(onehot_df, df, column_name, fill_na, drop_name):
    onehot_df[column_name] = df[column_name]
    if fill_na is not None:
        onehot_df[column_name].fillna(fill_na, inplace=True)

    dummies = pd.get_dummies(onehot_df[column_name], prefix='_' + column_name)
    
    # Dropping one of the columns actually made the results slightly worse.
    # if drop_name is not None:
    #     dummies.drop(['_' + column_name + '_' + drop_name], axis=1, inplace=True)

    onehot_df = onehot_df.join(dummies)
    onehot_df = onehot_df.drop([column_name], axis=1)
    return onehot_df

In [24]:
def munge_onehot(df):
    onehot_df = pd.DataFrame(index = df.index)

    onehot_df = onehot(onehot_df, df, 'MSSubClass', None, '40')
    onehot_df = onehot(onehot_df, df, 'MSZoning', 'RL', 'RH')
    onehot_df = onehot(onehot_df, df, 'LotConfig', None, 'FR3')
    onehot_df = onehot(onehot_df, df, 'Neighborhood', None, 'OldTown')
    onehot_df = onehot(onehot_df, df, 'Condition1', None, 'RRNe')
    onehot_df = onehot(onehot_df, df, 'BldgType', None, '2fmCon')
    onehot_df = onehot(onehot_df, df, 'HouseStyle', None, '1.5Unf')
    onehot_df = onehot(onehot_df, df, 'RoofStyle', None, 'Shed')
    onehot_df = onehot(onehot_df, df, 'Exterior1st', 'VinylSd', 'CBlock')
    onehot_df = onehot(onehot_df, df, 'Exterior2nd', 'VinylSd', 'CBlock')
    onehot_df = onehot(onehot_df, df, 'Foundation', None, 'Wood')
    onehot_df = onehot(onehot_df, df, 'SaleType', 'WD', 'Oth')
    onehot_df = onehot(onehot_df, df, 'SaleCondition', 'Normal', 'AdjLand')

    # Fill in missing MasVnrType for rows that do have a MasVnrArea.
    temp_df = df[['MasVnrType', 'MasVnrArea']].copy()
    idx = (df['MasVnrArea'] != 0) & ((df['MasVnrType'] == 'None') | (df['MasVnrType'].isnull()))
    temp_df.loc[idx, 'MasVnrType'] = 'BrkFace'
    onehot_df = onehot(onehot_df, temp_df, 'MasVnrType', 'None', 'BrkCmn')

    # Also add the booleans from calc_df as dummy variables.
    onehot_df = onehot(onehot_df, df, 'LotShape', None, 'IR3')
    onehot_df = onehot(onehot_df, df, 'LandContour', None, 'Low')
    onehot_df = onehot(onehot_df, df, 'LandSlope', None, 'Sev')
    onehot_df = onehot(onehot_df, df, 'Electrical', 'SBrkr', 'FuseP')
    onehot_df = onehot(onehot_df, df, 'GarageType', 'None', 'CarPort')
    onehot_df = onehot(onehot_df, df, 'PavedDrive', None, 'P')
    onehot_df = onehot(onehot_df, df, 'MiscFeature', 'None', 'Othr')

    # Features we can probably ignore (but want to include anyway to see
    # if they make any positive difference).
    # Definitely ignoring Utilities: all records are 'AllPub', except for
    # one 'NoSeWa' in the train set and 2 NA in the test set.
    onehot_df = onehot(onehot_df, df, 'Street', None, 'Grvl')
    onehot_df = onehot(onehot_df, df, 'Alley', 'None', 'Grvl')
    onehot_df = onehot(onehot_df, df, 'Condition2', None, 'PosA')
    onehot_df = onehot(onehot_df, df, 'RoofMatl', None, 'WdShake')
    onehot_df = onehot(onehot_df, df, 'Heating', None, 'Wall')

    # I have these as numerical variables too.
    onehot_df = onehot(onehot_df, df, 'ExterQual', 'None', 'Ex')
    onehot_df = onehot(onehot_df, df, 'ExterCond', 'None', 'Ex')
    onehot_df = onehot(onehot_df, df, 'BsmtQual', 'None', 'Ex')
    onehot_df = onehot(onehot_df, df, 'BsmtCond', 'None', 'Ex')
    onehot_df = onehot(onehot_df, df, 'HeatingQC', 'None', 'Ex')
    onehot_df = onehot(onehot_df, df, 'KitchenQual', 'TA', 'Ex')
    onehot_df = onehot(onehot_df, df, 'FireplaceQu', 'None', 'Ex')
    onehot_df = onehot(onehot_df, df, 'GarageQual', 'None', 'Ex')
    onehot_df = onehot(onehot_df, df, 'GarageCond', 'None', 'Ex')
    onehot_df = onehot(onehot_df, df, 'PoolQC', 'None', 'Ex')
    onehot_df = onehot(onehot_df, df, 'BsmtExposure', 'None', 'Gd')
    onehot_df = onehot(onehot_df, df, 'BsmtFinType1', 'None', 'GLQ')
    onehot_df = onehot(onehot_df, df, 'BsmtFinType2', 'None', 'GLQ')
    onehot_df = onehot(onehot_df, df, 'Functional', 'Typ', 'Typ')
    onehot_df = onehot(onehot_df, df, 'GarageFinish', 'None', 'Fin')
    onehot_df = onehot(onehot_df, df, 'Fence', 'None', 'MnPrv')
    onehot_df = onehot(onehot_df, df, 'MoSold', None, None)
    
    # Divide up the years between 1871 and 2010 in slices of 20 years.
    year_map = pd.concat(pd.Series('YearBin' + str(i+1), index=range(1871+i*20,1891+i*20)) for i in range(0, 7))

    yearbin_df = pd.DataFrame(index = df.index)
    yearbin_df['GarageYrBltBin'] = df.GarageYrBlt.map(year_map)
    # yearbin_df['GarageYrBltBin'].fillna('NoGarage', inplace=True)
    yearbin_df['GarageYrBltBin'].fillna('Unknown', inplace=True)

    yearbin_df['YearBuiltBin'] = df.YearBuilt.map(year_map)
    yearbin_df['YearRemodAddBin'] = df.YearRemodAdd.map(year_map)
    
    onehot_df = onehot(onehot_df, yearbin_df, 'GarageYrBltBin', None, None)
    onehot_df = onehot(onehot_df, yearbin_df, 'YearBuiltBin', None, None)
    onehot_df = onehot(onehot_df, yearbin_df, 'YearRemodAddBin', None, None)

    return onehot_df

In [25]:
# Add the one-hot encoded categorical features.
onehot_df = munge_onehot(train_filled)
onehot_df = onehot(onehot_df, neighborhood_bin_train, 'NeighborhoodBin', None, None)
train_onehot = train_transformed.join(onehot_df)

In [26]:
# These onehot columns are missing in the test data, so drop them from the
# training data or we might overfit on them.
drop_cols = [
                '_Exterior1st_ImStucc', '_Exterior1st_Stone',
                '_Exterior2nd_Other','_HouseStyle_2.5Fin', 
            
                '_RoofMatl_Membran', '_RoofMatl_Metal', '_RoofMatl_Roll',
                '_Condition2_RRAe', '_Condition2_RRAn', '_Condition2_RRNn',
                '_Heating_Floor', '_Heating_OthW',

                '_Electrical_Mix', 
                '_MiscFeature_TenC',
                '_GarageQual_Ex', '_PoolQC_Fa',
                '_RoofMatl_ClyTile'
            ]
train_onehot.drop(drop_cols, axis=1, inplace=True)

In [27]:
onehot_df = munge_onehot(test_filled)
onehot_df = onehot(onehot_df, neighborhood_bin_test, 'NeighborhoodBin', None, None)
test_onehot = test_transformed.join(onehot_df)

# This column is missing in the training data. There is only one example with
# this value in the test set. So just drop it.
test_onehot.drop(['_MSSubClass_150'], axis=1, inplace=True)

In [28]:
# Drop these columns. They are either not very helpful or they cause overfitting.
drop_cols = [
    '_Condition2_PosN',    # only two are not zero
    '_MSZoning_C (all)',
    '_MSSubClass_160',
]
# Realy overfit!
train_onehot.drop(drop_cols, axis=1, inplace=True)
test_onehot.drop(drop_cols, axis=1, inplace=True)

In [30]:
# Add a few columns to ensure train and test have the same shape
fake_in_train=['_Functional_NA', '_KitchenQual_NA', '_Exterior1st_Other', 
               '_GarageYrBltBin_Unknown', '_Exterior2nd_Other']
for f in fake_in_train:
    train_onehot[f]=0
fake_in_test=['_Electrical_NA']
for f in fake_in_test:
    test_onehot[f]=0
    
# Handle naming issue
train_onehot=train_onehot.rename(index=str, columns={"_NeighborhoodBin_0.0": "_NeighborhoodBin_0", "_NeighborhoodBin_1.0": "_NeighborhoodBin_1",
                                          "_NeighborhoodBin_2.0": "_NeighborhoodBin_2","_NeighborhoodBin_3.0": "_NeighborhoodBin_3",
                                          "_NeighborhoodBin_4.0": "_NeighborhoodBin_4"})



In [31]:
# The train and test set should by now have the same feature number. If not, do something.
print train_onehot.shape
print test_onehot.shape

(1460, 407)
(1459, 407)


In [32]:
# Check the features that differ between train and test set
test_columns=test_onehot.columns
print test_columns
train_columns=train_onehot.columns
print train_columns

print 'Only in test: ',
print set(test_columns)-set(train_columns)

print 'Only in train: ',
print set(train_columns)-set(test_columns)

Index([u'LotFrontage', u'LotArea', u'MasVnrArea', u'BsmtFinSF1', u'BsmtFinSF2',
       u'BsmtUnfSF', u'TotalBsmtSF', u'1stFlrSF', u'2ndFlrSF', u'GrLivArea',
       ...
       u'_YearRemodAddBin_YearBin4', u'_YearRemodAddBin_YearBin5',
       u'_YearRemodAddBin_YearBin6', u'_YearRemodAddBin_YearBin7',
       u'_NeighborhoodBin_0', u'_NeighborhoodBin_1', u'_NeighborhoodBin_2',
       u'_NeighborhoodBin_3', u'_NeighborhoodBin_4', u'_Electrical_NA'],
      dtype='object', length=407)
Index([u'LotFrontage', u'LotArea', u'MasVnrArea', u'BsmtFinSF1', u'BsmtFinSF2',
       u'BsmtUnfSF', u'TotalBsmtSF', u'1stFlrSF', u'2ndFlrSF', u'GrLivArea',
       ...
       u'_NeighborhoodBin_0', u'_NeighborhoodBin_1', u'_NeighborhoodBin_2',
       u'_NeighborhoodBin_3', u'_NeighborhoodBin_4', u'_Functional_NA',
       u'_KitchenQual_NA', u'_Exterior1st_Other', u'_GarageYrBltBin_Unknown',
       u'_Exterior2nd_Other'],
      dtype='object', length=407)
Only in test:  set([])
Only in train:  set([])


In [51]:
# Now it's time we concat the SalePrice column back to the training set

# Drop first in case we didn't remove this column
if 'SalePrice' in train_onehot.columns.names:
    print 'Drop SalePrice from training set first'
    train_onehot=train_onehot.drop(['SalePrice'],axis=1)

target_df=pd.DataFrame(train_target)
print 'y shape'
print type(train_target),train_target.shape,len(train_target)
print 'X shape'
print train_onehot.shape

y shape
<class 'pandas.core.series.Series'> (1460L,) 1460
X shape
(1460, 407)
After joining
(1460, 408)


In [50]:
# try join, no good
t=train_onehot.join(train_target)
t

,LotFrontage,LotArea,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,...,_NeighborhoodBin_1,_NeighborhoodBin_2,_NeighborhoodBin_3,_NeighborhoodBin_4,_Functional_NA,_KitchenQual_NA,_Exterior1st_Other,_GarageYrBltBin_Unknown,_Exterior2nd_Other,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,0.212877,-0.133270,1.203619,0.779431,-0.355342,-0.340511,0.002513,-0.803645,1.180528,0.529194,...,0.0,1.0,0.0,0.0,0,0,0,0,0,NaN
2,0.645747,0.113413,-0.806841,0.888257,-0.355342,0.002218,0.341114,0.418479,-0.870117,-0.381965,...,0.0,0.0,1.0,0.0,0,0,0,0,0,NaN
3,0.299451,0.420049,1.131524,0.654803,-0.355342,0.230372,0.065397,-0.576677,1.184762,0.659631,...,0.0,1.0,0.0,0.0,0,0,0,0,0,NaN
4,0.068587,0.103317,-0.806841,0.384539,-0.355342,0.348034,-0.105819,-0.439421,1.143550,0.541448,...,0.0,0.0,1.0,0.0,0,0,0,0,0,NaN
5,0.761179,0.878431,1.423411,0.754400,-0.355342,0.295711,0.256237,0.112127,1.244087,1.282295,...,0.0,0.0,0.0,1.0,0,0,0,0,0,NaN
6,0.790037,0.858672,-0.806841,0.791506,-0.355342,-0.795296,-0.060861,-1.032382,1.055765,-0.153271,...,0.0,1.0,0.0,0.0,0,0,0,0,0,NaN
7,0.501457,0.208507,1.183794,1.000604,-0.355342,0.061333,0.593849,1.345589,-0.870117,0.500996,...,0.0,0.0,1.0,0.0,0,0,0,0,0,NaN
8,-1.662893,0.264812,1.280332,0.844928,1.540387,-0.144859,0.226794,0.005860,1.223213,1.131148,...,0.0,1.0,0.0,0.0,0,0,0,0,0,NaN
9,-0.191135,-0.756945,-0.806841,-1.414140,-0.355342,0.653527,0.095218,-0.245674,1.141941,0.639409,...,1.0,0.0,0.0,0.0,0,0,0,0,0,NaN


In [55]:
# Try concat, no good
pd.concat([train_onehot,train_target],axis=1)

,LotFrontage,LotArea,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,...,_NeighborhoodBin_2,_NeighborhoodBin_3,_NeighborhoodBin_4,_Functional_NA,_KitchenQual_NA,_Exterior1st_Other,_GarageYrBltBin_Unknown,_Exterior2nd_Other,SalePrice,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,208500.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,181500.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,223500.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140000.0
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,250000.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,143000.0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,307000.0
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200000.0
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,129900.0


In [53]:
print len(train_onehot['SalePrice'])
print train_onehot['SalePrice'].shape

1460
(1460L,)


In [124]:
print train_final.shape
print test_onehot.shape
train_final['SalePrice'].describe()

(1460, 403)
(1459, 406)


count      1460.000000
mean     180921.195890
std       79442.502883
min       34900.000000
25%      129975.000000
50%      163000.000000
75%      214000.000000
max      755000.000000
Name: SalePrice, dtype: float64

In [33]:
# In fact I'm not able to concat the SalePrice column successfully 
# So don't expect this to work
# Just paste the column in excel, please

# Validate the y value are added
print train_target
print type(train_target),train_target.shape
print train_df_munged['SalePrice']

Id
1       208500
2       181500
3       223500
4       140000
5       250000
6       143000
7       307000
8       200000
9       129900
10      118000
11      129500
12      345000
13      144000
14      279500
15      157000
16      132000
17      149000
18       90000
19      159000
20      139000
21      325300
22      139400
23      230000
24      129900
25      154000
26      256300
27      134800
28      306000
29      207500
30       68500
         ...  
1431    192140
1432    143750
1433     64500
1434    186500
1435    160000
1436    174000
1437    120500
1438    394617
1439    149700
1440    197000
1441    191000
1442    149300
1443    310000
1444    121000
1445    179600
1446    129000
1447    157900
1448    240000
1449    112000
1450     92000
1451    136000
1452    287090
1453    145000
1454     84500
1455    185000
1456    175000
1457    210000
1458    266500
1459    142125
1460    147500
Name: SalePrice, dtype: int64
<class 'pandas.core.series.Series'> (1460L,)
Id
1   

In [38]:
# We cannot drop the rows for now as the Saleprice column needs some manual work

print 'Before dropping outliers ',
print train_df_munged.shape

# There are a few houses with more than 4000 sq ft living area that are
# outliers, so we drop them from the training data. (There is also one in
# the test set but we obviously can't drop that one.)
train_df_munged.drop(train_df_munged[train_df_munged['GrLivArea'] > 4000].index, inplace=True)

print 'After dropping outliers ',
print train_df_munged.shape

Before dropping outliers  (1460, 409)
After dropping outliers  (1460, 409)


In [96]:
# Output 
train_final.to_csv(base_path+'paragon_train.csv')
test_onehot.to_csv(base_path+'paragon_test.csv')